In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objs as go
import plotly.express as px
import joblib
import json

In [7]:
def is_in_box(row, box_min, box_max):
    for feat, min_val in box_min.items():
        if row.get(int(feat), 0) < min_val:
            return False
    for feat, max_val in box_max.items():
        if row.get(int(feat), 0) > max_val:
            return False
    return True

def generate_box_features(X: pd.DataFrame, box_data: dict) -> pd.DataFrame:
    box_features = pd.DataFrame(index=X.index)

    for box_id, box_limits in box_data.items():
        box_min = box_limits.get('min', {})
        box_max = box_limits.get('max', {})

        # For each row, determine if it satisfies the box conditions
        box_features[f'box_{box_id}'] = X.apply(lambda row: is_in_box(row, box_min, box_max), axis=1).astype(int)

    return box_features

In [11]:
# Custom model as fallback
class CustomModel:
    def __init__(self, road_importances):
        self.road_importances = road_importances
    
    def predict(self, X):
        X_array = X[0]  # Get the input array
        selected_indices = np.where(X_array > 0)[0]
        selected_roads = [idx for idx in selected_indices]
        
        # Base time + penalty for each closed road
        base_time = 3.0
        road_penalty = sum(self.road_importances.get(idx, 0.01) * 50 
                          for idx in selected_indices)
        
        return np.array([base_time + road_penalty])

In [45]:
# Build meta-model
def build_meta_model(df):
    # Separate features and target
    X = df.drop('evacuation_time', axis=1)
    y = df['evacuation_time']
    
    # Check if there's enough variation in the target
    if y.std() < 0.1:
        print("WARNING: Not enough variation in evacuation times. Adding synthetic data...")
        # Create some synthetic data points
        for i in range(20):
            # Generate random road closures
            row = np.zeros(len(X.columns))
            num_closed = np.random.randint(2, 10)
            closed_indices = np.random.choice(len(X.columns), num_closed, replace=False)
            row[closed_indices] = 1
            
            # Generate reasonable evacuation time (10-60 minutes)
            evacuation_time = 10 + np.random.rand() * 50
            
            # Add to training data
            X = pd.concat([X, pd.DataFrame([row], columns=X.columns)])
            y = pd.concat([y, pd.Series([evacuation_time])])

    # Train model
    model = RandomForestRegressor(n_estimators=100, min_samples_leaf=2, random_state=42)
    model.fit(X, y)
    
    # Verify model works by testing different inputs
    test_input = np.zeros(X.shape[1])
    baseline = model.predict([test_input])[0]
    
    test_input[0] = 1  # Close first road
    pred1 = model.predict([test_input])[0]
    
    test_input[0] = 0
    test_input[10] = 1  # Close a different road
    pred2 = model.predict([test_input])[0]
    
    print(f"Model test - Baseline: {baseline}, Pred1: {pred1}, Pred2: {pred2}")
    #if abs(baseline - pred1) < 0.01 and abs(baseline - pred2) < 0.01:
    #    print("WARNING: Model not responding to different inputs. Returning a custom model...")
    #    return CustomModel(road_importances)

    return model

In [16]:
import pandas as pd
import numpy as np
import random

# Create sample data
def generate_sample_scenarios(num_scenarios=500, num_roads=50):
    # Initialize dataframe columns
    columns = [f'road_{i}' for i in range(num_roads)]
    columns.append('evacuation_time')
    
    # Create empty dataframe
    df = pd.DataFrame(columns=columns)
    
    # Generate scenarios
    for i in range(num_scenarios):
        # For each scenario, randomly close 5-20 roads
        num_closed_roads = random.randint(5, 20)
        road_closures = np.zeros(num_roads)
        
        # Randomly select roads to close
        closed_road_indices = random.sample(range(num_roads), num_closed_roads)
        road_closures[closed_road_indices] = 1
        
        # Simple model for evacuation time:
        # - Base time: 60 minutes
        # - Each closed road adds 2-10 minutes depending on importance
        # - Some combinations have extra penalties
        road_importance = np.random.exponential(scale=1.0, size=num_roads)
        road_importance = road_importance / np.max(road_importance)  # Normalize to 0-1
        
        base_time = 60
        road_effect = np.sum(road_closures * road_importance * 10)
        
        # Add some interaction effects for certain combinations
        key_road_pairs = [(10, 25), (50, 100), (200, 300), (400, 500)]
        interaction_penalty = 0
        for r1, r2 in key_road_pairs:
            if road_closures[r1] == 1 and road_closures[r2] == 1:
                interaction_penalty += 15
        
        # Add noise
        noise = np.random.normal(0, 5)
        
        evacuation_time = base_time + road_effect + interaction_penalty + noise
        
        # Create row
        row = list(road_closures) + [evacuation_time]
        df.loc[i] = row
    
    return df

# Generate sample data
scenarios_df = generate_sample_scenarios(500, 2200)

# Save to CSV (saving just a subset of columns for demonstration)
# In reality, you'd save all columns
sample_df = scenarios_df.iloc[:, list(range(0, 50)) + [-1]]
sample_df.to_csv('evacuation_scenarios_sample.csv', index=False)

print("Sample data generated and saved to 'evacuation_scenarios_sample.csv'")

KeyboardInterrupt: 

In [ ]:
def generate_feature_importance_file(num_roads=50):
    # Generate mock feature importance values
    # In real life, these would come from your model analysis
    importances = np.random.exponential(scale=1.0, size=num_roads)
    importances = importances / np.sum(importances)  # Normalize to sum to 1
    
    # Create dataframe
    importance_df = pd.DataFrame({
        'road_id': [f'road_{i}' for i in range(num_roads)],
        'importance': importances
    })
    
    # Sort by importance
    importance_df = importance_df.sort_values('importance', ascending=False)
    
    # Save to CSV
    importance_df.to_csv('road_importance.csv', index=False)
    
    print("Feature importance file generated and saved to 'road_importance.csv'")

generate_feature_importance_file()

Feature importance file generated and saved to 'road_importance.csv'


In [ ]:
def generate_scenario_discovery_results():
    # This would represent the output from your scenario discovery analysis
    # Format: combinations of roads and their impact on evacuation time
    
    scenarios = [
        {
            'scenario_id': 1,
            'description': 'Major arterial routes blocked',
            'conditions': [
                {'road_id': 'road_10', 'status': 'closed'},
                {'road_id': 'road_25', 'status': 'closed'},
                {'road_id': 'road_42', 'status': 'closed'}
            ],
            'avg_evacuation_time': 112.5,
            'probability_high_impact': 0.87,
            'coverage': 0.12  # Portion of high-impact scenarios covered by this rule
        },
        {
            'scenario_id': 2,
            'description': 'Southern exit routes blocked',
            'conditions': [
                {'road_id': 'road_10', 'status': 'closed'},
                {'road_id': 'road_16', 'status': 'closed'},
                {'road_id': 'road_17', 'status': 'closed'}
            ],
            'avg_evacuation_time': 98.3,
            'probability_high_impact': 0.75,
            'coverage': 0.08
        },
        {
            'scenario_id': 3,
            'description': 'Downtown grid blocked',
            'conditions': [
                {'road_id': 'road_25', 'status': 'closed'},
                {'road_id': 'road_21', 'status': 'closed'},
                {'road_id': 'road_22', 'status': 'closed'},
                {'road_id': 'road_15', 'status': 'closed'}
            ],
            'avg_evacuation_time': 105.2,
            'probability_high_impact': 0.82,
            'coverage': 0.15
        }
    ]
    
    # Save to JSON
    import json
    with open('scenario_discovery_results.json', 'w') as f:
        json.dump(scenarios, f, indent=4)
    
    print("Scenario discovery results saved to 'scenario_discovery_results.json'")

generate_scenario_discovery_results()

Scenario discovery results saved to 'scenario_discovery_results.json'


In [ ]:
def generate_road_network_info():
    # This would contain information about your road network
    # For visualization purposes
    
    roads = []
    
    # Generate some random roads with coordinates
    # In reality, this would be your actual road network
    for i in range(2200):
        road = {
            'road_id': f'road_{i}',
            'start_x': random.uniform(0, 100),
            'start_y': random.uniform(0, 100),
            'end_x': random.uniform(0, 100),
            'end_y': random.uniform(0, 100),
            'capacity': random.randint(500, 2000),  # vehicles per hour
            'road_type': random.choice(['arterial', 'collector', 'local']),
            'zone': random.choice(['north', 'south', 'east', 'west', 'central'])
        }
        roads.append(road)
    
    # Save to CSV
    road_df = pd.DataFrame(roads)
    road_df.to_csv('road_network.csv', index=False)
    
    print("Road network information saved to 'road_network.csv'")

generate_road_network_info()

Road network information saved to 'road_network.csv'


In [9]:
### evacScenarioFile, roadImportanceFile, scenarioDiscoveryFile, roadNetworkFile
def load_data():
    """
    Load all necessary data files for the evacuation visualization tool
    
    Returns:
        df: DataFrame containing road closure scenarios
        road_importances: Dictionary mapping road IDs to their importance scores
        scenario_rules: List of discovered scenario patterns
        road_network: DataFrame containing road network information
    """
    # Load scenario data
    try:
        #df = pd.read_csv('evacuation_scenarios_sample.csv')
        df = pd.read_csv('metaModel/ladris_df.csv')
        print(f"Loaded {len(df)} scenario records with {len(df.columns)-1} road features")
    except FileNotFoundError:
        print("Warning: Evacuation scenarios file not found. Creating sample data...")
        # Create a small sample dataset if file doesn't exist
        num_roads = 50  # Using a small subset for demonstration
        columns = [f'road_{i}' for i in range(num_roads)]
        columns.append('evacuation_time')
        
        df = pd.DataFrame(columns=columns)
        for i in range(df.columns):  # 100 sample scenarios
            # Generate random road closures (1 = closed, 0 = open)
            road_closures = np.zeros(num_roads)
            num_closed = np.random.randint(2, 10)
            closed_indices = np.random.choice(num_roads, num_closed, replace=False)
            road_closures[closed_indices] = 1
            
            # Generate evacuation time (simple model: base time + effect of closures)
            base_time = 60
            # More effect for lower-numbered roads (assumed to be more important)
            importance_weights = np.exp(-np.arange(num_roads)/10)
            evacuation_time = base_time + np.sum(road_closures * importance_weights * 20)
            evacuation_time += np.random.normal(0, 5)  # Add noise
            
            row = list(road_closures) + [evacuation_time]
            df.loc[i] = row
    
    # Load feature importance data
    try:
        #importance_df = pd.read_csv('road_importance.csv')
        importance_df = pd.read_csv('metaModel/ladris_feature_importance.csv')
        road_importances = dict(zip(importance_df['road_id'], importance_df['importance']))
        print(f"Loaded importance scores for {len(road_importances)} roads")
    except FileNotFoundError:
        print("Warning: Road importance file not found. Generating from scenario data...")
        # If feature importance file doesn't exist, calculate it from the scenario data
        # using a simple Random Forest model
        from sklearn.ensemble import RandomForestRegressor
        
        X = df.drop('evacuation_time', axis=1)
        y = df['evacuation_time']
        
        model = RandomForestRegressor(n_estimators=50, random_state=42)
        model.fit(X, y)
        
        importances = model.feature_importances_
        road_importances = {f'road_{i}': imp for i, imp in enumerate(importances)}

    ### Load ladris_conv2_scenarioDiscovery.json
    try:
        #with open('scenario_discovery_results.json', 'r') as f:
        with open('metaModel/ladris_conv2_scenarioDiscovery.json', 'r') as f:
            scenario_rules = json.load(f)
        print(f"Loaded {len(scenario_rules)} scenario patterns")
    except FileNotFoundError:
        print("Warning: Scenario discovery file not found. Creating sample scenarios...")

    X_augmented = pd.concat([df, generate_box_features(df, scenario_rules)], axis=1)
    
    # Load scenario discovery results
    try:
        #with open('scenario_discovery_results.json', 'r') as f:
        with open('metaModel/ladris_scenario_discovery.json', 'r') as f:
            scenario_rules = json.load(f)
        print(f"Loaded {len(scenario_rules)} scenario patterns")
    except FileNotFoundError:
        print("Warning: Scenario discovery file not found. Creating sample scenarios...")
        # Create some sample scenario rules if file doesn't exist
        scenario_rules = [
            {
                'scenario_id': 1,
                'description': 'Major arterial routes blocked',
                'conditions': [
                    {'road_id': 'road_0', 'status': 'closed'},
                    {'road_id': 'road_1', 'status': 'closed'}
                ],
                'avg_evacuation_time': 95.0,
                'probability_high_impact': 0.85,
                'coverage': 0.25
            },
            {
                'scenario_id': 2,
                'description': 'Northern exit routes blocked',
                'conditions': [
                    {'road_id': 'road_5', 'status': 'closed'},
                    {'road_id': 'road_10', 'status': 'closed'}
                ],
                'avg_evacuation_time': 88.5,
                'probability_high_impact': 0.75,
                'coverage': 0.20
            }
        ]
    
    # Load road network information
    try:
        #road_network = pd.read_csv('road_network.csv')
        road_network = pd.read_csv('metaModel/ladris_latlong.csv')
        print(f"Loaded network data for {len(road_network)} roads")
    except FileNotFoundError:
        print("Warning: Road network file not found. Creating sample network...")
        # Create a sample road network if file doesn't exist
        road_network = pd.DataFrame(columns=[
            'road_id', 'start_x', 'start_y', 'end_x', 'end_y', 
            'capacity', 'road_type', 'zone'
        ])
        
        # Get the road IDs from the scenario data
        road_ids = [col for col in df.columns if col.startswith('road_')]
        
        # Create a grid-like network
        grid_size = int(np.sqrt(len(road_ids)))
        for i, road_id in enumerate(road_ids):
            # Determine road position in grid
            row = i // grid_size
            col = i % grid_size
            
            # Determine if horizontal or vertical road
            is_horizontal = (i % 2 == 0)
            
            if is_horizontal:
                start_x = col * 10
                start_y = row * 10
                end_x = (col + 1) * 10
                end_y = row * 10
            else:
                start_x = col * 10
                start_y = row * 10
                end_x = col * 10
                end_y = (row + 1) * 10
            
            # Assign road type based on position (central roads more important)
            center = grid_size // 2
            dist_from_center = max(abs(row - center), abs(col - center))
            if dist_from_center <= grid_size // 5:
                road_type = 'arterial'
                capacity = np.random.randint(1500, 2000)
            elif dist_from_center <= grid_size // 3:
                road_type = 'collector'
                capacity = np.random.randint(1000, 1500)
            else:
                road_type = 'local'
                capacity = np.random.randint(500, 1000)
            
            # Assign zone based on position
            if row < grid_size // 2 and col < grid_size // 2:
                zone = 'northwest'
            elif row < grid_size // 2 and col >= grid_size // 2:
                zone = 'northeast'
            elif row >= grid_size // 2 and col < grid_size // 2:
                zone = 'southwest'
            else:
                zone = 'southeast'
            
            # Add to dataframe
            road_network.loc[i] = [
                road_id, start_x, start_y, end_x, end_y,
                capacity, road_type, zone
            ]
    
    return X_augmented, road_importances, scenario_rules, road_network

In [ ]:

import json

# Load your data and build the model
print("Loading data...")
df, road_importances, scenario_rules, road_network = load_data()



Loading data...
Loaded 500 scenario records with 50 road features
Loaded importance scores for 2200 roads
Loaded 3 scenario patterns
Loaded network data for 2200 roads


In [ ]:
print("Building model...")
model = build_meta_model(df)



Building model...


In [ ]:
print("Creating dashboard...")
app = create_dashboard(model, road_importances, scenario_rules, road_network)



Creating dashboard...


In [ ]:
print("Running dashboard...")
app.run(debug=True)

Running dashboard...


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[29], line 73, in create_dashboard.<locals>.update_output(
    n_clicks=1,
    selected_roads=['road_660', 'road_1967', 'road_387', 'road_606']
)
     70         X_input[road_idx] = 1
     72 # Predict evacuation time
---> 73 pred_time = model.predict([X_input])[0]
        X_input = array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
        [X_input] = [array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]
        model = (RandomForestRegressor(random_state=42), {'road_0': 0.008931960911302361, '

In [ ]:
import os
import signal

# Find the process ID
# This example works for Linux/Mac
# For Windows, you might need a different approach
try:
    pid = int(os.popen('lsof -i:8050 -t').read().strip())
    os.kill(pid, signal.SIGTERM)
    print(f"Killed Dash server process {pid}")
except:
    print("Could not find or kill the Dash server process")

: 

In [39]:
def build_improved_meta_model(df):
    # Separate features and target
    X = df.drop('evacuation_time', axis=1)
    y = df['evacuation_time']
    
    # Add feature engineering - calculate road density/importance metrics
    importance_scores = np.zeros(X.shape[1])
    for i in range(X.shape[1]):
        temp_X = X.copy()
        temp_X.iloc[:, i] = 1  # Close this road
        model_temp = RandomForestRegressor(n_estimators=10)
        model_temp.fit(temp_X, y)
        importance_scores[i] = model_temp.feature_importances_[i]
    
    # Add these feature importance scores as weights
    weighted_X = X.copy()
    for i in range(X.shape[1]):
        weighted_X.iloc[:, i] = X.iloc[:, i] * (importance_scores[i] + 0.1)
    
    # Train a gradient boosting model instead
    from sklearn.ensemble import GradientBoostingRegressor
    model = GradientBoostingRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=4,
        min_samples_leaf=3,
        random_state=42
    )
    model.fit(weighted_X, y)
    
    # Verify model performance
    test_scores = cross_val_score(model, weighted_X, y, cv=5)
    print(f"Cross-validation scores: {test_scores.mean():.2f} ± {test_scores.std():.2f}")
    
    return model, importance_scores

In [40]:
def make_improved_prediction(model, importance_scores, input_data):
    # Convert input to appropriate format
    if isinstance(input_data, pd.DataFrame):
        X = input_data.copy()
    else:
        X = pd.DataFrame([input_data], columns=range(len(input_data)))
    
    # Apply importance weighting
    for i in range(X.shape[1]):
        X.iloc[:, i] = X.iloc[:, i] * (importance_scores[i] + 0.1)
    
    # Make prediction
    prediction = model.predict(X)
    
    # Add baseline travel time (could be data-driven)
    baseline = 2.0  # Minimum evacuation time with no closures
    
    # Scale prediction based on number of closed roads
    num_closed = X.sum(axis=1).values[0]
    scaling_factor = 1.0 + (num_closed / 100)  # Adjust formula as needed
    
    return max(baseline, prediction * scaling_factor)

In [46]:
# Main execution
if __name__ == '__main__':
    print("Loading data...")
    ### evacScenarioFile, roadImportanceFile, scenarioDiscoveryFile, roadNetworkFile
    df, road_importances, scenario_rules, road_network = load_data()#"ladris_df.csv", )
    
    ### check if evacuation_model.pkl exists
    try:
        model = joblib.load('metaModel/evacuation_model.pkl')
        print("Model loaded from evacuation_model.pkl")
    except FileNotFoundError:
        print("No existing model found. Building a new one...")
        # Build a new model
        print("Building model...")
        model = build_meta_model(df)    
        ### export model
        # Save the model to a file    
        joblib.dump(model, 'metaModel/evacuation_model.pkl')
        print("Model saved as evacuation_model.pkl")

    print("Creating dashboard...")
    #app = create_dashboard(model, road_importances, scenario_rules, road_network)
    
    print("Running dashboard...")
    #app.run(debug=True)

Loading data...
Loaded 10994 scenario records with 2295 road features
Loaded importance scores for 2295 roads
Loaded 10 scenario patterns
Loaded 1985 scenario patterns
Loaded network data for 2295 roads
No existing model found. Building a new one...
Building model...
Model test - Baseline: 3.8616162436849852, Pred1: 3.8577328876127637, Pred2: 3.8651671311545983
Model saved as evacuation_model.pkl
Creating dashboard...
Running dashboard...


/home/pfoytik/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/pfoytik/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/pfoytik/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [15]:
### make a prediction with model
def make_prediction(model, input_data):
    """
    Make a prediction using the trained model.
    
    Parameters:
        model: Trained model
        input_data: DataFrame containing the input data for prediction
    
    Returns:
        prediction: Predicted evacuation time
    """
    # Ensure input data is in the correct format
    if isinstance(input_data, pd.DataFrame):
        input_data = input_data.values
    
    print(input_data)
    # Make prediction
    prediction = model.predict(input_data)
    
    return prediction



In [30]:
# Example usage
road_input = np.zeros(df.shape[1])

# Example input data (this should match the feature set used for training)
example_input = pd.DataFrame({
    'road_0': [1],
    'road_1': [0],
    'road_2': [1],
    # Add more features as needed
})

### get column index of df for feature 330221, 340133, and 329968
feature_indices = [330221, 340133, 329968]
feature_indices = [df.columns.get_loc(f'{i}') for i in feature_indices]
print(f"Feature indices: {feature_indices}")

### create an example input where feature 330221, 340133, and 329968 are set to 1
example_input = np.zeros(df.shape[1])
for idx in feature_indices:
    example_input[idx] = 1
example_input = pd.DataFrame([example_input], columns=df.columns)
print(f"Example input: {example_input}")

# Make prediction
prediction = make_prediction(model, example_input)
print(f"Predicted evacuation time: {prediction[0]} hours")

Feature indices: [106, 1081, 1087]
Example input:    328559  330552  331652  331795  332555  332843  333389  333943  334433  \
0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

   334569  ...  box_0  box_1  box_2  box_3  box_4  box_5  box_6  box_7  box_8  \
0     0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

   box_9  
0    0.0  

[1 rows x 2306 columns]
[[0. 0. 0. ... 0. 0. 0.]]
Predicted evacuation time: 4.5 hours


In [50]:
testDF = df.drop('evacuation_time', axis=1)
### Run ten tests with 10 random features set to 1
for i in range(10):
    # Generate random input with 10 features set to 1
    random_input = np.zeros(testDF.shape[1])
    random_indices = np.random.choice(testDF.shape[1], 10, replace=False)
    ### print the random indices
    print(f"Random indices: {random_indices}")
    for idx in random_indices:
        random_input[idx] = 1
    random_input = pd.DataFrame([random_input], columns=testDF.columns)
    
    # Make prediction
    prediction = make_prediction(model, random_input)
    print(f"Test {i+1}: Predicted evacuation time: {prediction[0]} hours")



Random indices: [ 805  800  105 2130 1185 1101 1463 2062 1839 2027]
[[0. 0. 0. ... 0. 0. 0.]]
Test 1: Predicted evacuation time: 3.9114966450995103 hours
Random indices: [1594   85    6  369 1539 1735 1319   48  715 2119]
[[0. 0. 0. ... 0. 0. 0.]]
Test 2: Predicted evacuation time: 3.9462226107715255 hours
Random indices: [1034 1185 1573 2285  329  826 1643  778 1735    7]
[[0. 0. 0. ... 0. 0. 0.]]
Test 3: Predicted evacuation time: 3.871733152601856 hours
Random indices: [ 728  299 1385 2001  656 2163 1728 1618  454 1280]
[[0. 0. 0. ... 0. 0. 0.]]
Test 4: Predicted evacuation time: 3.973831343697361 hours
Random indices: [1631  144 2236 1624 1759 1655   40 1644  746 2243]
[[0. 0. 0. ... 0. 0. 0.]]
Test 5: Predicted evacuation time: 3.9704152857398123 hours
Random indices: [ 355 2239 1377 2189 1504 1492  403 1141 2016  559]
[[0. 0. 0. ... 0. 0. 0.]]
Test 6: Predicted evacuation time: 3.8004829328971397 hours
Random indices: [ 320 2043 1881    1 1420 1560 1460  212 1094 1853]
[[0. 1. 0.

/home/pfoytik/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/pfoytik/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/pfoytik/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/pfoytik/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/pfoytik/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegress

[[0. 0. 0. ... 0. 0. 0.]]
Test 9: Predicted evacuation time: 3.96080793138822 hours
Random indices: [2151  427 1151 1893 1326  110  414  475  297 1543]
[[0. 0. 0. ... 0. 0. 0.]]
Test 10: Predicted evacuation time: 3.926831182422479 hours


/home/pfoytik/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/home/pfoytik/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [35]:
### calculate the standard deviation of df['evacuation_time']
df['evacuation_time'].std()

1.6429423164238657

In [41]:
model